<a href="https://colab.research.google.com/github/mukulsn/Projects/blob/main/RecSys/Notebooks/RecSys_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Date - 19 jan 2025

this notebook facilitates to get the data for H&M recomendation system.
from paul lusztin course.

but because paul uses Hopsworks but for me hopsworks is not working so i am changing the code. changing the root code of his course to avoid using hopsworks. and making it altogether new code wherever required.

In [ ]:
!pip install boto3==1.26.138


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0


In [ ]:
import os
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''

# new way

In [ ]:
import boto3
from botocore.exceptions import ClientError

# Replace with your AWS credentials
AWS_ACCESS_KEY_ID = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = os.environ['AWS_SECRET_ACCESS_KEY']
BUCKET_NAME = "recsys-system"

# Create an S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    Args:
        file_name (str): File to upload
        bucket (str): Bucket to upload to
        object_name (str): S3 object name. If not specified then file_name is used
    """
    if object_name is None:
        object_name = file_name

    try:
        response = s3.upload_file(file_name, bucket, object_name)
        print(f"Uploaded {file_name} to {bucket}/{object_name}")
    except ClientError as e:
        print(f"Error uploading {file_name} to {bucket}/{object_name}: {e}")

def download_file(bucket, object_name, file_name):
    """Download a file from an S3 bucket

    Args:
        bucket (str): Bucket to download from
        object_name (str): S3 object name
        file_name (str): File to download to
    """
    try:
        s3.download_file(bucket, object_name, file_name)
        print(f"Downloaded {object_name} from {bucket} to {file_name}")
    except ClientError as e:
        print(f"Error downloading {object_name} from {bucket}: {e}")

def delete_object(bucket, object_name):
    """Delete an object from an S3 bucket

    Args:
        bucket (str): Bucket to delete from
        object_name (str): S3 object name
    """
    try:
        response = s3.delete_object(Bucket=bucket, Key=object_name)
        print(f"Deleted {object_name} from {bucket}")
    except ClientError as e:
        print(f"Error deleting {object_name} from {bucket}: {e}")

def upload_dir_to_s3(local_dir, bucket_name, s3_prefix):
    """
    Uploads the contents of a local directory to an S3 bucket with a specified prefix.

    Args:
        local_dir: Path to the local directory to upload.
        bucket_name: Name of the S3 bucket.
        s3_prefix: Prefix for the objects in the S3 bucket.
    """
    s3_client = boto3.client('s3')

    for root, dirs, files in os.walk(local_dir):
        for file_name in files:
            local_file_path = os.path.join(root, file_name)
            relative_path = os.path.relpath(local_file_path, local_dir)
            s3_key = os.path.join(s3_prefix, relative_path)

            try:
                s3_client.upload_file(local_file_path, bucket_name, s3_key)
                print(f"Uploaded {local_file_path} to s3://{bucket_name}/{s3_key}")
            except Exception as e:
                print(f"Error uploading {local_file_path}: {e}")


# Example usage
# upload_file('local_file.txt', BUCKET_NAME)
# download_file(BUCKET_NAME, 'remote_file.txt', 'local_file.txt')
# delete_object(BUCKET_NAME, 'remote_file.txt')

In [ ]:
# download_file(BUCKET_NAME, 'data/articles.parquet', 'articles_df.parquet')
download_file(BUCKET_NAME, 'data/customers.parquet', 'customers.parquet')
download_file(BUCKET_NAME, 'data/transactions.parquet', 'transactions.parquet')




Downloaded data/customers.parquet from recsys-system to customers.parquet
Downloaded data/transactions.parquet from recsys-system to transactions.parquet


In [ ]:
import polars as po
articles_df = po.read_parquet('/content/articles_df.parquet')
customers_df = po.read_parquet('/content/customers.parquet')
transactions_df = po.read_parquet('/content/transactions.parquet')

# Actual code

In [ ]:

import time

notebook_start_time = time.time()

In [ ]:
import sys
from pathlib import Path


def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False


def clone_repository() -> None:
    !git clone https://github.com/decodingml/hands-on-recommender-system.git
    %cd hands-on-recommender-system/


def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()

    root_dir = str(Path().absolute())
    print("⛳️ Google Colab environment")
else:
    root_dir = str(Path().absolute().parent)
    print("⛳️ Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

Cloning into 'hands-on-recommender-system'...
remote: Enumerating objects: 927, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 927 (delta 41), reused 50 (delta 27), pack-reused 837 (from 1)
Receiving objects: 100% (927/927), 4.38 MiB | 15.01 MiB/s, done.
Resolving deltas: 100% (565/565), done.
/content/hands-on-recommender-system
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 31.1 MB/s eta 0:00:00
Using Python 3.11.11 environment at: /usr
Resolved 198 packages in 3.65s
Building avro==1.11.3
Building avro==1.11.3
⠙ Preparing packages... (0/53)
Building avro==1.11.3
⠙ Preparing packages... (0/53)
Building avro==1.11.3
⠙ Preparing packages... (0/53)
jmespath   ------------------------------ 14.83 KiB/19.78 KiB
Building avro==1.11.3
⠙ Preparing packages... (0/53)
jmespath   ------------------------------ 14.83 KiB/19.78 KiB
Building avro==1.11.3
⠙ Preparing packages... (0/53)
jmespath   --------------------------

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings

warnings.filterwarnings("ignore")

from loguru import logger

from recsys import hopsworks_integration, training
from recsys.config import settings

In [ ]:
from pprint import pprint

pprint(dict(settings))

{'CUSTOMER_DATA_SIZE': <CustomerDatasetSize.SMALL: 'SMALL'>,
 'CUSTOM_HOPSWORKS_INFERENCE_ENV': 'custom_env_name',
 'FEATURES_EMBEDDING_MODEL_ID': 'all-MiniLM-L6-v2',
 'HOPSWORKS_API_KEY': None,
 'OPENAI_API_KEY': None,
 'OPENAI_MODEL_ID': 'gpt-4o-mini',
 'RANKING_DATASET_VALIDATON_SPLIT_SIZE': 0.1,
 'RANKING_EARLY_STOPPING_ROUNDS': 5,
 'RANKING_ITERATIONS': 100,
 'RANKING_LEARNING_RATE': 0.2,
 'RANKING_MODEL_TYPE': 'ranking',
 'RANKING_SCALE_POS_WEIGHT': 10,
 'RECSYS_DIR': PosixPath('/content/hands-on-recommender-system/recsys'),
 'TWO_TOWER_DATASET_TEST_SPLIT_SIZE': 0.1,
 'TWO_TOWER_DATASET_VALIDATON_SPLIT_SIZE': 0.1,
 'TWO_TOWER_LEARNING_RATE': 0.01,
 'TWO_TOWER_MODEL_BATCH_SIZE': 2048,
 'TWO_TOWER_MODEL_EMBEDDING_SIZE': 16,
 'TWO_TOWER_NUM_EPOCHS': 10,
 'TWO_TOWER_WEIGHT_DECAY': 0.001}


In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
from pprint import pprint

import polars as pl
import torch
from loguru import logger
from sentence_transformers import SentenceTransformer

warnings.filterwarnings("ignore")

from recsys import hopsworks_integration
from recsys.config import settings
from recsys.features.articles import (
    compute_features_articles,
    generate_embeddings_for_dataframe,
)
from recsys.features.customers import DatasetSampler, compute_features_customers
from recsys.features.interaction import generate_interaction_data
from recsys.features.ranking import compute_ranking_dataset
from recsys.features.transactions import compute_features_transactions
from recsys.hopsworks_integration import feature_store
from recsys.raw_data_sources import h_and_m as h_and_m_raw_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
articles_df = h_and_m_raw_data.extract_articles_df()
# customers_df = h_and_m_raw_data.extract_customers_df()
# transactions_df = h_and_m_raw_data.extract_transactions_df()

In [ ]:
articles_df = compute_features_articles(articles_df)
articles_df.shape

(105542, 27)

In [ ]:
for i, desc in enumerate(articles_df["article_description"].head(n=3)):
    logger.info(f"Item {i+1}:\n{desc}")

2025-01-18 20:11:22.771 | INFO     | __main__:<cell line: 0>:2 - Item 1:
Strap top - Vest top in Garment Upper body
Appearance: Solid
Color: Dark Black (Black)
Category: Ladieswear - Womens Everyday Basics - Jersey Basic
Details: Jersey top with narrow shoulder straps.
2025-01-18 20:11:22.784 | INFO     | __main__:<cell line: 0>:2 - Item 2:
Strap top - Vest top in Garment Upper body
Appearance: Solid
Color: Light White (White)
Category: Ladieswear - Womens Everyday Basics - Jersey Basic
Details: Jersey top with narrow shoulder straps.
2025-01-18 20:11:22.788 | INFO     | __main__:<cell line: 0>:2 - Item 3:
Strap top (1) - Vest top in Garment Upper body
Appearance: Stripe
Color: Dusty Light White (Off White)
Category: Ladieswear - Womens Everyday Basics - Jersey Basic
Details: Jersey top with narrow shoulder straps.


In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'cuda'

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
logger.info(
    f"Loading '{settings.FEATURES_EMBEDDING_MODEL_ID}' embedding model to {device=}"
)

# Load the embedding model from SentenceTransformer's model registry.
model = SentenceTransformer(settings.FEATURES_EMBEDDING_MODEL_ID, device=device)

2025-01-18 20:11:23.191 | INFO     | __main__:<cell line: 0>:8 - Loading 'all-MiniLM-L6-v2' embedding model to device='cuda'


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
articles_df = generate_embeddings_for_dataframe(
    articles_df, "article_description", model, batch_size=128
)  # Reduce ba

Generating embeddings:   0%|          | 0/105542 [00:00<?, ?it/s]

In [ ]:
articles_df.write_parquet('/content/articles.parquet')

In [ ]:
upload_file('/content/articles.parquet', BUCKET, object_name='features/articles.parquet')

NameError: name 'upload_file' is not defined

In [ ]:
# as of now uploading features data to gdrive

In [ ]:
!cp '/content/articles.parquet' '/content/drive/MyDrive/Machine Learning/Paul Lusztin/RecSys/data/'

# customers

In [ ]:
customers_df = h_and_m_raw_data.extract_customers_df()
customers_df.shape

(1371980, 7)

In [ ]:
customers_df = compute_features_customers(customers_df, drop_null_age=True)
customers_df.shape

(1356119, 5)

In [ ]:
customers_df.write_parquet('/content/customers.parquet')

In [ ]:
!cp '/content/customers.parquet' '/content/drive/MyDrive/Machine Learning/Paul Lusztin/RecSys/data/'

# transactions data

In [ ]:
transactions_df = h_and_m_raw_data.extract_transactions_df()
transactions_df.shape

(31788324, 5)

In [ ]:
transactions_df = compute_features_transactions(transactions_df)
transactions_df.shape

(31788324, 9)

In [ ]:
transactions_df.write_parquet('/content/transactions.parquet')

In [ ]:
!cp '/content/transactions.parquet' '/content/drive/MyDrive/Machine Learning/Paul Lusztin/RecSys/data/'

# the above files saved in gdrive, will now be used below this cell. because for calculation of interaction data and ranking data they have taken the subset only not the full dataset

In [ ]:
sampler = DatasetSampler(size=settings.CUSTOMER_DATA_SIZE)
dataset_subset = sampler.sample(
    customers_df=customers_df, transations_df=transactions_df
)
customers_df = dataset_subset["customers"]
transactions_df = dataset_subset["transactions"]

2025-01-18 21:12:46.510 | INFO     | recsys.features.customers:sample:29 - Sampling 1000 customers.
2025-01-18 21:12:46.762 | INFO     | recsys.features.customers:sample:32 - Number of transactions for all the customers: 31788324
2025-01-18 21:12:51.277 | INFO     | recsys.features.customers:sample:38 - Number of transactions for the 1000 sampled customers: 23799


In [ ]:
transactions_df.shape

(23799, 9)

In [ ]:
for customer_id in transactions_df["customer_id"].unique().head(10):
    logger.info(f"Logging customer ID: {customer_id}")

2025-01-18 21:13:16.625 | INFO     | __main__:<cell line: 0>:2 - Logging customer ID: fdc791c174a6aee7afe354daa679229e5e4e2efb665d57c08159df7129cadfff
2025-01-18 21:13:16.629 | INFO     | __main__:<cell line: 0>:2 - Logging customer ID: 035b13a68d68ecf01309c0a80f9f081071142706669acfe1fe9043c5b8a2d74c
2025-01-18 21:13:16.631 | INFO     | __main__:<cell line: 0>:2 - Logging customer ID: 9cb1f79e0fb899fbc0dcd1064915c99fc64ef583b4b7d819fd3941d6c2fd56b4
2025-01-18 21:13:16.634 | INFO     | __main__:<cell line: 0>:2 - Logging customer ID: 74a212b12c377b678d9d5454c8c95dc2e2cc84dd4d219f5bca061c460af3554a
2025-01-18 21:13:16.636 | INFO     | __main__:<cell line: 0>:2 - Logging customer ID: 32f6112378b205e601fa55fd174399473577b309cc39046086d01a0cb2f5aaf1
2025-01-18 21:13:16.639 | INFO     | __main__:<cell line: 0>:2 - Logging customer ID: f6b2a7e9b2fd05134f3fbe746ce7de1e696fcd051612b431f6a836025efe8fe1
2025-01-18 21:13:16.641 | INFO     | __main__:<cell line: 0>:2 - Logging customer ID: fa2f14b7

In [ ]:
interaction_df = generate_interaction_data(transactions_df)
interaction_df.shape

Processing customer chunks: 100%|██████████| 1/1 [00:10<00:00, 10.18s/it]


(135774, 5)

In [ ]:
interaction_df.group_by("interaction_score").agg(
    pl.count("interaction_score").alias("total_interactions")
)

interaction_score,total_interactions
i64,u32
0,73727
1,38248
2,23799


In [ ]:
interaction_df.write_parquet('/content/interaction.parquet')

In [ ]:
!cp '/content/interaction.parquet' '/content/drive/MyDrive/Machine Learning/Paul Lusztin/RecSys/data/'

In [ ]:
interaction_df.head()

t_dat,customer_id,article_id,interaction_score,prev_article_id
i64,str,str,i64,str
-367200000,"""00b203a32faa3d007dba198ef27c15…","""844198001""",0,"""START"""
-363600000,"""00b203a32faa3d007dba198ef27c15…","""746648003""",0,"""844198001"""
-352800000,"""00b203a32faa3d007dba198ef27c15…","""892051004""",0,"""746648003"""
-349200000,"""00b203a32faa3d007dba198ef27c15…","""844198001""",0,"""892051004"""
-345600000,"""00b203a32faa3d007dba198ef27c15…","""745475011""",0,"""844198001"""


In [ ]:
customers_df.shape

(1000, 5)

In [ ]:
def compute_ranking_dataset_new(trans_fg, articles_fg, customers_fg) -> pl.DataFrame:
    # Read data from the feature groups
    trans_df = trans_fg.select(
        ["article_id", "customer_id"]
    )
    articles_df = articles_fg.select(pl.exclude(["article_description", "embeddings", "image_url"]))
    customers_df = customers_fg.select(["customer_id", "age"])

    # Convert article_id to string in both dataframes before joining
    trans_df = trans_df.with_columns(pl.col("article_id").cast(pl.Utf8))
    articles_df = articles_df.with_columns(pl.col("article_id").cast(pl.Utf8))

    # Merge operations
    df = trans_df.join(articles_df, on="article_id", how="left")
    df = df.join(customers_df, on="customer_id", how="left")

    # Select query features
    query_features = ["customer_id", "age", "article_id"]
    df = df.select(query_features)

    # Create positive pairs
    positive_pairs = df.clone()

    # Calculate number of negative pairs
    n_neg = len(positive_pairs) * 10

    # Create negative pairs DataFrame
    article_ids = (df.select("article_id")
                    .unique()
                    .sample(n=n_neg, with_replacement=True, seed=2)
                    .get_column("article_id"))

    customer_ids = (df.select("customer_id")
                     .sample(n=n_neg, with_replacement=True, seed=3)
                     .get_column("customer_id"))

    other_features = (df.select(["age"])
                       .sample(n=n_neg, with_replacement=True, seed=4))

    # Construct negative pairs
    negative_pairs = pl.DataFrame({
        "article_id": article_ids,
        "customer_id": customer_ids,
        "age": other_features.get_column("age"),
    })

    # Add labels
    positive_pairs = positive_pairs.with_columns(pl.lit(1).alias("label"))
    negative_pairs = negative_pairs.with_columns(pl.lit(0).alias("label"))

    # Concatenate positive and negative pairs
    ranking_df = pl.concat([
        positive_pairs,
        negative_pairs.select(positive_pairs.columns)
    ])

    # Process item features
    item_df = articles_fg

    # Convert article_id to string in item_df before final join
    item_df = item_df.with_columns(pl.col("article_id").cast(pl.Utf8))

    # Keep unique article_ids and select columns
    item_df = (
        item_df.unique(subset=["article_id"])
        .select([
            "article_id",
            "product_type_name",
            "product_group_name",
            "graphical_appearance_name",
            "colour_group_name",
            "perceived_colour_value_name",
            "perceived_colour_master_name",
            "department_name",
            "index_name",
            "index_group_name",
            "section_name",
            "garment_group_name",
        ])
    )

    # Final merge with item features
    ranking_df = ranking_df.join(item_df, on="article_id", how="left")

    return ranking_df

In [ ]:
ranking_df = compute_ranking_dataset_new(
    transactions_df,
    articles_df,
    customers_df,
)
ranking_df.shape

(261789, 15)

In [ ]:
ranking_df.get_column("label").value_counts()

label,count
i32,u32
1,23799
0,237990


In [ ]:
ranking_df.write_parquet('/content/ranking.parquet')

In [ ]:
!cp '/content/ranking.parquet' '/content/drive/MyDrive/Machine Learning/Paul Lusztin/RecSys/data/'

In [ ]:
ranking_df

customer_id,age,article_id,label,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,index_group_name,section_name,garment_group_name
str,f64,str,i32,str,str,str,str,str,str,str,str,str,str,str
"""25f0bcb3b9e6fffee7541b02d21e48…",22.0,"""541518004""",1,"""Bra""","""Underwear""","""Solid""","""Black""","""Dark""","""Black""","""Casual Lingerie""","""Lingeries/Tights""","""Ladieswear""","""Womens Lingerie""","""Under-, Nightwear"""
"""25f0bcb3b9e6fffee7541b02d21e48…",22.0,"""654564002""",1,"""Bra""","""Underwear""","""Solid""","""White""","""Light""","""White""","""Expressive Lingerie""","""Lingeries/Tights""","""Ladieswear""","""Womens Lingerie""","""Under-, Nightwear"""
"""25f0bcb3b9e6fffee7541b02d21e48…",22.0,"""372860002""",1,"""Socks""","""Socks & Tights""","""Solid""","""White""","""Light""","""White""","""Shopbasket Socks""","""Lingeries/Tights""","""Ladieswear""","""Womens Nightwear, Socks & Tigh""","""Socks and Tights"""
"""4e82363f3c5a710922073cdf626309…",49.0,"""673285001""",1,"""Sweater""","""Garment Upper body""","""Stripe""","""White""","""Light""","""White""","""Jersey Fancy""","""Menswear""","""Menswear""","""Contemporary Smart""","""Jersey Fancy"""
"""4e82363f3c5a710922073cdf626309…",49.0,"""564314018""",1,"""Underwear bottom""","""Underwear""","""All over pattern""","""Dark Blue""","""Medium""","""Blue""","""Underwear Jersey""","""Menswear""","""Menswear""","""Men Underwear""","""Under-, Nightwear"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""5d34f84e6cbe9ec4706872bb653760…",30.0,"""622958013""",0,"""Sneakers""","""Shoes""","""Solid""","""White""","""Light""","""White""","""Sneakers""","""Ladies Accessories""","""Ladieswear""","""Womens Shoes""","""Shoes"""
"""91a30617b9a5b5ff3927779204a176…",26.0,"""901281001""",0,"""Trousers""","""Garment Lower body""","""Dot""","""Black""","""Dark""","""Black""","""Trousers""","""Divided""","""Divided""","""Divided Collection""","""Trousers"""
"""973d446f084a9242592d01f9b59699…",28.0,"""751271002""",0,"""Shorts""","""Garment Lower body""","""Solid""","""Black""","""Dark""","""Black""","""Jersey""","""Ladieswear""","""Ladieswear""","""H&M+""","""Jersey Fancy"""


# uploading files to s3

In [ ]:
!cp '/content/drive/MyDrive/Machine Learning/Paul Lusztin/RecSys/data/articles.parquet' '/content/'

In [ ]:
upload_file('/content/articles.parquet', 'recsys-system', object_name='features/articles.parquet')

Uploaded /content/articles.parquet to recsys-system/features/articles.parquet


In [ ]:
!cp '/content/drive/MyDrive/Machine Learning/Paul Lusztin/RecSys/data/customers.parquet' '/content/'

In [ ]:
def custom_upload(file):
  '''
  file : it is the name of the file only not the extension of file.
  '''
  print(upload_file(f'/content/{file}.parquet', 'recsys-system', object_name=f'features/{file}.parquet'))

In [ ]:
custom_upload('customers')

Uploaded /content/customers.parquet to recsys-system/features/customers.parquet
None


In [ ]:
!cp '/content/drive/MyDrive/Machine Learning/Paul Lusztin/RecSys/data/transactions.parquet' '/content/'

In [ ]:
custom_upload('transactions')

Uploaded /content/transactions.parquet to recsys-system/features/transactions.parquet
None


In [ ]:
!cp '/content/drive/MyDrive/Machine Learning/Paul Lusztin/RecSys/data/ranking.parquet' '/content/'
custom_upload('ranking')

Uploaded /content/ranking.parquet to recsys-system/features/ranking.parquet
None


In [ ]:
!cp '/content/drive/MyDrive/Machine Learning/Paul Lusztin/RecSys/data/interaction.parquet' '/content/'
custom_upload('interaction')

Uploaded /content/interaction.parquet to recsys-system/features/interaction.parquet
None
